<a href="https://colab.research.google.com/github/mille-s/DCU_TCD-FORGe_WebNLG23/blob/main/DCU_TCD_FORGe_WebNLG23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0a - Download resources and install Java 8

In [1]:
# Run this cell to download and unzip the working folder (once run, click "Refresh" on the top left corner to see the folders)
from IPython.display import clear_output

# Download FORGe
! gdown 1lsh8pwUp9mc0Z_aFbSy1WTIpSx9YwFFD
! unzip /content/FORGe_colab_v3.zip

# Download Morphology
! gdown 1vk1utEjeZ_2YO1H20DPDTjVSevgRJNM_
folder_name = 'test_irish_morph_gen_v5.0'
zip_name = folder_name+'.zip'
! unzip {zip_name}

import os
morph_input_folder = '/content/'+folder_name+'/Inputs'
morph_output_folder = '/content/'+folder_name+'/Outputs'
os.makedirs(morph_input_folder)
os.makedirs(morph_output_folder)

# Make flookup executable
! 7z a -sfx {folder_name}'/flookup.exe' {folder_name}'/flookup'
! chmod 755 {folder_name}'/flookup'

# Clean
! rm '/content/FORGe_colab_v3.zip'
! rm '/content/test_irish_morph_gen_v5.0.zip'
clear_output()

In [2]:
# Run this cell to switch to Java 1.8 (needed for FORGe to run correctly)
import os
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
  !java -version       #check java version
install_java()

update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
openjdk version "1.8.0_382"
OpenJDK Runtime Environment (build 1.8.0_382-8u382-ga-1~22.04.1-b05)
OpenJDK 64-Bit Server VM (build 25.382-b05, mixed mode)


# 0b - Set parameters

In [3]:
# PARAMETERS FOR GENERATION

# The input structure(s) of the correct type should be placed in the folder that corresponds to the first module called in the next cell
# E.g. if one a module PredArg_... or DSynt_... is selected, the input predicate-argument structures should be placed in the structures/00-PredArg folder
# I'll make the instructions and names clearer in a later (actually usable) version.

######################################################################

# Select language
language = 'GA' #@param['EN', 'ES', 'FR', 'GA']

#######################################################################

# Group consecutive modules for the same system or call each module separately.
# Select 'no' to get all intermediate representations, 'yes' if you're only interested in the output.
group_modules_prm = 'yes' #@param['yes', 'no']

#######################################################################

# Modules to run, with type of processing (FORGe, Model1, SimpleNLG, etc.).
# Only FORGe is supported for this prototype version.
# What if a module spans over several of these?
PredArg_Normalisation = 'FORGe'
PredArg_AggregationMark = ''
PredArg_Aggregation = 'FORGe'
PredArg_PoSTagging = 'FORGe'
PredArg_CommStructuring = 'FORGe'
DSynt_Structuring = 'FORGe'
SSynt_Structuring = 'FORGe'
SSynt_Aggregation = 'FORGe'
RE_Generation = 'FORGe'
DMorph_AgreementsLinearisation = 'FORGe'
SMorph_Processing = 'FORGe'
# Define all micro modules and several higher level modules that can overlap, the highest level being the one-shot generation.
#Surface_Generation = 'IMS' # That could take DSynt/SSynt as input and return text; to be defined during the query processing

# 1 - RDF to PredArg (TBD)

# 2 - PredArg to uninflected text (FORGe via M-FleNS pipeline code)

## FORGe Static parameters and functions

In [4]:
# No need to edit this block for running the generator

# The elements of this list are referenced from several places in the code
# When updating list, update process_input function!
#                 0             1               2                3             4            5            6        7          8        9         10          11
level_names = ['PredArg', 'PredArgNorm', 'PredArgAggMark', 'PredArgAgg', 'PredArgPoS', 'PredArgComm', 'DSynt', 'SSynt', 'SSyntAgg', 'REG', 'DMorphLin', 'SMorphText']

# Static - Description of RGB modules (FORGe)
# Also add dicos here to save some time when loading resources
PredArg0_dict = {'input': ['Init'], 'grammars': []}
PredArg1_Normalisation_dict = {'input': [level_names[0]], 'grammars': ['10_Con_Sem.rl']}
PredArg2_Aggregation_dict = {'input': [level_names[1], level_names[2]], 'grammars': ['11.1_Con_Agg1.rl', '11.2_Con_Agg2.rl', '11.3_Con_Agg3.rl', '11.4_Con_Agg4.rl']}
PredArg3_PoSTagging_dict = {'input': [level_names[1], level_names[3]], 'grammars': ['13_Sem_SemPoS.rl']}
PredArg4_CommStructuring_dict = {'input': [level_names[4]], 'grammars': ['15_SemPoS_SemCommMark.rl', '17_SemCommMark_SemComm.rl']}
DSynt_Structuring_dict = {'input': [level_names[5]], 'grammars': ['20_SemComm_DSynt.rl']}
SSynt_Structuring_dict = {'input': [level_names[6]], 'grammars': ['30_DSynt_SSynt.rl', '35_SSynt_PostProc.rl']}
SSynt_Aggregation_dict = {'input': [level_names[7]], 'grammars': ['37.1_SSynt_Agg1.rl', '37.2_SSynt_Agg2.rl']}
REG_dict = {'input': [level_names[7], level_names[8]], 'grammars': ['38.1_SSynt_REG1.rl', '38.2_SSynt_REG2.rl']}
DMorph_AgreementsLinearisation_dict = {'input': [level_names[7], level_names[8], level_names[9]], 'grammars': ['40_SSynt_DMorph_linearize.rl']}
SMorph_Processing_dict = {'input': [level_names[10]], 'grammars': ['50_DMorph_SMorph.rl', '60_Smorph_Sentence.rl']}

# Static - Description of RGB dicos (FORGe)
dic_common_list = ['EN_control.dic', 'concepticon.dic', 'EN_lexicon_MS.dic', 'project_KRISTINA.dic']
dic_indep_list = ['language_info.dic', 'lexicon.dic', 'semanticon.dic', 'morphologicon.dic']
# If a lexicon.dic, semanticon.dic or morphologicon.dic has something else in the name beyond the language prefix, put in this dictionary
dic_special_name_dico = {'EN_lexicon.dic':'EN_lexicon_SMALL.dic'}

# Paths to FORGe/MATE folders and property files
FORGe_input_folder = '/content/FORGe/buddy_project/struct'
path_MATE = '/content/FORGe/buddy-patched.jar'
path_props_resources_template = '/content/FORGe/mateColabDrive.properties'
path_props_levels = '/content/FORGe/mateLevels.properties'
path_props = '/content/FORGe/mate.properties'

# Paths to general folders
# The input structure(s) of the correct type should be placed in the folder that corresponds to the first module called in the next cell
str_PredArg_folder = '/content/FORGe/structures/00-PredArg'
str_PredArgNorm_folder = '/content/FORGe/structures/01-PredArgNorm'
str_PredArgAggMark_folder = '/content/FORGe/structures/02-PredArgAggMark'
str_PredArgAgg_folder = '/content/FORGe/structures/03-PredArgAgg'
str_PredArgPoS_folder = '/content/FORGe/structures/04-PredArgPoS'
str_PredArgComm_folder = '/content/FORGe/structures/05-PredArgComm'
str_DSynt_folder = '/content/FORGe/structures/06-DSynt'
str_SSynt_folder = '/content/FORGe/structures/07-SSynt'
str_SSyntAgg_folder = '/content/FORGe/structures/08-SSyntAgg'
str_REG_folder = '/content/FORGe/structures/09-REG'
str_DMorphLin_folder = '/content/FORGe/structures/10-DMorphLin'
str_SMorphText_folder = '/content/FORGe/structures/11-SMorphText'

class RGBModule:
  "Class to store information related to the RGB modules"
  def __init__(self, module, system, dico_module, in_folder, out_folder):
    self.module_type = 'RGB'
    self.system = system
    self.output = str(module)
    self.inputs = dico_module.get('input')
    self.grammars = dico_module.get('grammars')
    self.input_folder = in_folder
    self.output_folder = out_folder

def clear_files(folder):
  "Function to clear files from a folder."
  if os.path.exists(folder) and os.path.isdir(folder):
    for filename in os.listdir(folder):
      file_path = os.path.join(folder, filename)
      try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
          os.unlink(file_path)
        elif os.path.isdir(file_path):
          shutil.rmtree(file_path)
      except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

def clear_folder(folder):
  "Function to clear whole folders."
  if os.path.exists(folder) and os.path.isdir(folder):
    for subfolder in os.listdir(folder):
      folder_path = os.path.join(folder, subfolder)
      if os.path.isdir(folder_path):
        try:
          shutil.rmtree(folder_path)
        except Exception as e:
          print('Failed to delete %s. Reason: %s' % (folder, e))

def rename_files(folder, output_type):
  "Function that renames files into more a human-friendly format."
  str_folder_content = os.listdir(folder)
  for filename in str_folder_content:
    filepath_to_change = os.path.join(folder, filename)
    filename_extension = filename.rsplit('.', 1)[1]
    filename_noExt = filename.rsplit('.', 1)[0]
    clean_filename_noExt = filename_noExt.rsplit('__')[0]
    new_filename = clean_filename_noExt+'__'+output_type+'.'+filename_extension
    new_filepath = os.path.join(folder, new_filename)
    os.rename(filepath_to_change, new_filepath)

def copy_files(pathIn, pathOut):
  "Function to copy files into the folder they need to be in to be processed."
  #print(pathIn)
  str_folder_content = os.listdir(pathIn)
  for content in str_folder_content:
      target_path = os.path.join(pathIn, content)
      # If files are found in the folder, copy files to FORGe's input folder
      if os.path.isfile(target_path):
        shutil.copy(target_path, pathOut)
      # If folders are found in the folder, explore them to get to the files
      elif os.path.isdir(target_path):
        str_subfolder_content = os.listdir(target_path)
        for deeper_content in str_subfolder_content:
          new_target_path = os.path.join(target_path, deeper_content)
          if os.path.isfile(new_target_path):
            shutil.copy(new_target_path, pathOut)

def check_pipeline(list_modules):
  "Function that finds the first structure to be processed by the pipeline."
  output_str = []
  input_str = []
  init_str = []
  for module_object in list_modules:
    # Output_str is a string
    output_str.append(module_object.output)
    # Input_str is a list that contains one or more elements
    input_str.append(module_object.inputs)
  for list_input_level in input_str:
    list_seen = []
    # For a given module, check for how many input types we find a corresponding structure to process
    for unique_input_level in list_input_level:
      if unique_input_level in output_str:
        list_seen.append(unique_input_level)
    # If none of the candidate input types is found, the module is the first to apply in the pipeline
    if len(list_seen) == 0:
      init_str.append(list_input_level)
  # If there is only one list of candidate inputs, that's OK, but if there is none or several, it means the pipeline is wrong.
  if len(init_str) == 1:
    print('  -> Initial structure is '+str(init_str[0])+'.')
    return(init_str[0])
  else:
    print('! ERROR! Hole in the pipeline, several possible first modules, or mapping not defined in "process_query" function for a system ('+str(len(init_str))+' initial representations found: '+str(init_str)+').')
    exit()

def define_module_sequence(list_modules):
  "Function that creates all possible module sequences and returns the one that include all modules"
  candidate_output_sequence_list = []
  # Find first module to apply
  for module_object in list_modules:
    # At this point we made sure that only one module expects the initial structure type (see check_pipeline function)
    if module_object.inputs == list_initial_str:
      candidate_output_sequence_list.append([module_object.output])

  # Creates all possible sequences of modules and saves them as lists
  for candidate_output_sequence in candidate_output_sequence_list:
    for module_object in list_modules:
      for input_type in module_object.inputs:
        if input_type == candidate_output_sequence[-1]:
          new_candidate_output_sequence_list = candidate_output_sequence.copy()
          new_candidate_output_sequence_list.append(module_object.output)
          # Update initial list to keep the loop going
          candidate_output_sequence_list.append(new_candidate_output_sequence_list)
  # Look for the longest sequence of modules
  best_candidate_sequence = []
  for candidate_output_sequence in candidate_output_sequence_list:
    if len(candidate_output_sequence) > len(best_candidate_sequence):
      best_candidate_sequence = candidate_output_sequence
  return(best_candidate_sequence)

def group_modules(module_sequence, list_modules):
  "Groups consecutive modules that are run by the same system. module_sequence is a list of strings, list_modules a list of objects of class 'module'"
  # Let's make a list that will contain X lists for X different system sub-pipelines; the system name is the first element of each list, the second element is another list that contains the module object.
  # grouped_modules looks like this: [['SystemName1', [<Module1-Object>, <Module2-Object>]], ['SystemName2', [<Module3-Object>, <Module4-Object>]]]
  grouped_modules = []
  # the elements in module_sequence are in order of execution
  for output_level in module_sequence:
      for module_object in list_modules:
          if output_level == module_object.output:
            # Check that the first element of the last list is not the same as the currently examined object's system
            if len(grouped_modules) > 0:
              if len(grouped_modules[-1]) > 0:
                if grouped_modules[-1][0] == module_object.system:
                  # If the last list has the same system name as the current object, add the object to the group
                  grouped_modules[-1][1].append(module_object)
                else:
                  # Otherwise create the next list for the new system
                  grouped_modules.append([module_object.system, [module_object]])
              else:
                grouped_modules.append([module_object.system, [module_object]])
            else:
              grouped_modules.append([module_object.system, [module_object]])
  return(grouped_modules)

def create_grouped_module_objects_FORGe(system_modules):
  "Creates a new object of class RGB module that combines the consecutive modules. system_modules contains at least 2 modules."
  first_module = system_modules[0]
  last_module = system_modules[-1]
  grammars_list = []
  for module in system_modules:
    grammars_list = grammars_list + module.grammars
  grouped_dico = {'input': [first_module.inputs], 'grammars': grammars_list}
  grouped_module = RGBModule(last_module.output, 'FORGe', grouped_dico, first_module.input_folder, last_module.output_folder)
  return(grouped_module)

# def check_pipeline(modules, list_init_str):
# #  "Checks if there are holes in the pipeline (i.e. if an expected input structure is not provided by any other module. Is this one usefule now? CF find_initial_structure."
#   list_outputs = []
#   # build list of all outputs pruduced by the different modules
#   for module in modules:
#     if module.output not in list_outputs:
#       list_outputs.append(module.output)
#   # check that each module takes as input at least one of the outputs of the other modules
#   for module in modules:
#     list_input_seen = []
#     for input in module.inputs:
#       if input in list_outputs or input in list_init_str:
#        list_input_seen.append(input)
#     if len(list_input_seen) == 0:
#      print('ERROR! Incomplete pipeline: there will be no available '+str(input)+' structure (required by '+str(module.output)+' module).')
#      exit()

def process_query(list_modules, PredArg_Normalisation, PredArg_AggregationMark, PredArg_Aggregation, PredArg_PoSTagging, PredArg_CommStructuring, DSynt_Structuring, SSynt_Structuring, SSynt_Aggregation, RE_Generation, DMorph_AgreementsLinearisation, SMorph_Processing):
  "Function to parse the input parameters and prepare the generation pipeline"
  # For each module, create an object with all the relevant class info (module_name, system, dico_module, in_folder, out_folder)
  if PredArg_Normalisation == 'FORGe':
    PredArg_Normalisation_RGB = RGBModule(level_names[1], 'FORGe', PredArg1_Normalisation_dict, str_PredArg_folder, str_PredArgNorm_folder)
    list_modules.append(PredArg_Normalisation_RGB)
  else:
    pass
  if PredArg_AggregationMark == 'FORGe':
    pass
  else:
    pass
  if PredArg_Aggregation == 'FORGe':
    PredArg_Aggregation_RGB = RGBModule(level_names[3], 'FORGe', PredArg2_Aggregation_dict, str_PredArgNorm_folder, str_PredArgAgg_folder)
    list_modules.append(PredArg_Aggregation_RGB)
    if PredArg_PoSTagging == 'FORGe':
      PredArg_PoSTagging_RGB = RGBModule(level_names[4], 'FORGe', PredArg3_PoSTagging_dict, str_PredArgAgg_folder, str_PredArgPoS_folder)
      list_modules.append(PredArg_PoSTagging_RGB)
    elif PredArg_PoSTagging == 'HiddenFORGe':
      PredArg_PoSTagging_RGB = RGBModule(level_names[4], 'HiddenFORGe', PredArg3_PoSTagging_dict, str_PredArgAgg_folder, str_PredArgPoS_folder)
      list_modules.append(PredArg_PoSTagging_RGB)
    else:
      pass
  elif PredArg_PoSTagging == 'FORGe':
    PredArg_PoSTagging_RGB = RGBModule(level_names[4], 'FORGe', PredArg3_PoSTagging_dict, str_PredArgNorm_folder, str_PredArgPoS_folder)
    list_modules.append(PredArg_PoSTagging_RGB)
  else:
    pass
  if PredArg_CommStructuring == 'FORGe':
    PredArg_CommStructuring_RGB = RGBModule(level_names[5], 'FORGe', PredArg4_CommStructuring_dict, str_PredArgPoS_folder, str_PredArgComm_folder)
    list_modules.append(PredArg_CommStructuring_RGB)
  elif PredArg_CommStructuring == 'HiddenFORGe':
    PredArg_CommStructuring_RGB = RGBModule(level_names[5], 'HiddenFORGe', PredArg4_CommStructuring_dict, str_PredArgPoS_folder, str_PredArgComm_folder)
    list_modules.append(PredArg_CommStructuring_RGB)
  else:
    pass
  if DSynt_Structuring == 'FORGe':
    DSynt_Structuring_RGB = RGBModule(level_names[6], 'FORGe', DSynt_Structuring_dict, str_PredArgComm_folder, str_DSynt_folder)
    list_modules.append(DSynt_Structuring_RGB)
  else:
    pass
  if SSynt_Structuring == 'FORGe':
    SSynt_Structuring_RGB = RGBModule(level_names[7], 'FORGe', SSynt_Structuring_dict, str_DSynt_folder, str_SSynt_folder)
    list_modules.append(SSynt_Structuring_RGB)
  else:
    pass
  # Agg + REG + Lin
  if SSynt_Aggregation == 'FORGe':
    SSynt_Aggregation_RGB = RGBModule(level_names[8], 'FORGe', SSynt_Aggregation_dict, str_SSynt_folder, str_SSyntAgg_folder)
    list_modules.append(SSynt_Aggregation_RGB)
    # Agg + REG + Lin
    if RE_Generation == 'FORGe':
      REG_RGB = RGBModule(level_names[9], 'FORGe', REG_dict, str_SSyntAgg_folder, str_REG_folder)
      list_modules.append(REG_RGB)
      # Agg + REG + Lin
      if DMorph_AgreementsLinearisation == 'FORGe':
        DMorph_AgreementsLinearisation_RGB = RGBModule(level_names[10], 'FORGe', DMorph_AgreementsLinearisation_dict, str_REG_folder, str_DMorphLin_folder)
        list_modules.append(DMorph_AgreementsLinearisation_RGB)
      else:
        pass
    # Agg + Lin
    elif DMorph_AgreementsLinearisation == 'FORGe':
      DMorph_AgreementsLinearisation_RGB = RGBModule(level_names[10], 'FORGe', DMorph_AgreementsLinearisation_dict, str_SSyntAgg_folder, str_DMorphLin_folder)
      list_modules.append(DMorph_AgreementsLinearisation_RGB)
    else:
      pass
  # REG + Lin
  elif RE_Generation == 'FORGe':
    REG_RGB = RGBModule(level_names[9], 'FORGe', REG_dict, str_SSynt_folder, str_REG_folder)
    list_modules.append(REG_RGB)
    # REG + Lin
    if DMorph_AgreementsLinearisation == 'FORGe':
      DMorph_AgreementsLinearisation_RGB = RGBModule(level_names[10], 'FORGe', DMorph_AgreementsLinearisation_dict, str_REG_folder, str_DMorphLin_folder)
      list_modules.append(DMorph_AgreementsLinearisation_RGB)
  # Lin
  elif DMorph_AgreementsLinearisation == 'FORGe':
    DMorph_AgreementsLinearisation_RGB = RGBModule(level_names[10], 'FORGe', DMorph_AgreementsLinearisation_dict, str_SSynt_folder, str_DMorphLin_folder)
    list_modules.append(DMorph_AgreementsLinearisation_RGB)
  else:
    pass
  if SMorph_Processing == 'FORGe':
    SMorph_Processing_RGB = RGBModule(level_names[11], 'FORGe', SMorph_Processing_dict, str_DMorphLin_folder, str_SMorphText_folder)
    list_modules.append(SMorph_Processing_RGB)
  else:
    pass

def process_files_FORGe(module_object):
  "Function to call FORGe to process an input of level X and create an output of level Y"
  "The function must be called process_files_SystemName"

  # Erase files from input folder
  print('  Cleared input folder...')
  clear_files(FORGe_input_folder)

  # Generate mate.properties file, which points to the resources to apply
  print('  Generated mate.properties file...')
  input_folder_gen = module_object.input_folder
  output_folder_gen = module_object.output_folder
  grammars = module_object.grammars
  # Create list of dictionaries for generation
  dic_resources_list = []
  dic_resources_list.extend(dic_common_list)
  # Lang. info, Lexicon, semanticon, morphologicon
  # We add the language-specific prefix and check if there's a special name for this dico
  for dic_name in dic_indep_list:
    lspec_dic_name = language+'_'+dic_name
    if lspec_dic_name in dic_special_name_dico:
      dic_resources_list.append(dic_special_name_dico.get(lspec_dic_name))
    else:
      dic_resources_list.append(lspec_dic_name)

  # Delete existing property file
  if os.path.exists(path_props):
    os.remove(path_props)

  # Read template to create a new file
  prop_template = open(path_props_resources_template, 'r')
  lines_prop_template = prop_template.readlines()

  # Create a new property file
  with codecs.open(path_props, 'w', 'utf-8') as f:
      for line in lines_prop_template:
        if line.startswith('projectDir='):
          project_dir = FORGe_input_folder.rsplit('/', 1)[0]
          f.write('projectDir='+str(project_dir)+'\n')
        elif line.startswith('resources='):
          f.write('resources=')
          x = 0
          # All but last dico are followed by a comma
          while x < len(dic_resources_list) - 1:
            f.write(dic_resources_list[x])
            f.write(', ')
            x = x + 1
          # Last dico is followed by a linebreak
          if x == len(dic_resources_list) - 1:
            f.write(dic_resources_list[x])
            f.write('\n')
        elif line.startswith('ruleSets='):
            f.write('ruleSets=')
            x = 0
            while x < len(grammars) - 1:
              f.write(grammars[x])
              f.write(', ')
              x = x + 1
            if x == len(grammars) - 1:
              f.write(grammars[x])
              f.write('\n')
        elif line.startswith('outputDir='):
            f.write('outputDir='+output_folder_gen+'\n')
        elif line.startswith('generateText='):
          if output_folder_gen == str_SMorphText_folder:
            f.write('generateText=true'+'\n')
          else:
            f.write('generateText=false'+'\n')
        else:
          f.write(line)
  f.close()

  # Copy files into input folder
  print('  Copied files to input folder...')
  copy_files(input_folder_gen, FORGe_input_folder)

  # Rename files
  print('  Renamed files in input folder...')
  rename_files(FORGe_input_folder, module_object.output)

  # Run generator
  print('  Running module(s)...')
  !java -jar {path_MATE} {path_props} {path_props_levels} #-> "log.txt"

def process_files_HiddenFORGe(module_object):
  "Fake function to test the use of several systems in the pipeline"
  "The function must be called process_files_SystemName"
  process_files_FORGe(module_object)

## FORGe Main code

In [5]:
import os
import os.path
import subprocess
import sys
from sys import exit
import shutil
import codecs
import re
import random

!java -version
print('=================================================')

# Erase files from output folders
print('Clearing output folders...')
clear_folder(str_PredArgNorm_folder)
clear_folder(str_PredArgAggMark_folder)
clear_folder(str_PredArgAgg_folder)
clear_folder(str_PredArgPoS_folder)
clear_folder(str_PredArgComm_folder)
clear_folder(str_DSynt_folder)
clear_folder(str_SSynt_folder)
clear_folder(str_SSyntAgg_folder)
clear_folder(str_REG_folder)
clear_folder(str_DMorphLin_folder)
clear_folder(str_SMorphText_folder)

# Fill list_modules with module descriptions as objects of a particular Class (e.g. RGB), each of them with associated properties (e.g. input/output types and folders, grammars, etc.)
print('Preparing generation pipeline...')
list_modules = []
process_query(list_modules, PredArg_Normalisation, PredArg_AggregationMark, PredArg_Aggregation, PredArg_PoSTagging, PredArg_CommStructuring, DSynt_Structuring, SSynt_Structuring, SSynt_Aggregation, RE_Generation, DMorph_AgreementsLinearisation, SMorph_Processing)

# Just to make sure the code doesn't depend on the nice order of the list (currently the order established in the process_query function)
random.shuffle(list_modules)

# Find which structure the pipeline starts with, and check for holes in pipeline at the same time
list_initial_str = check_pipeline(list_modules)
print('  -> '+str(len(list_modules))+' modules were selected.')

# Find module sequence that includes all required modules in the right order
module_sequence = define_module_sequence(list_modules)
print('  -> Sequence: '+str(module_sequence))
if len(list_modules) == len(module_sequence):
  print('  -> The pipeline looks good, proceeding...')
else:
  print('! Possible ERROR! There are '+str(len(list_modules))+' modules selected and the longest module sequence found is '+str(len(module_sequence))+'.')

# Break pipeline by system, so we can make one call per system instead of one call per module.
modules_to_process = group_modules(module_sequence, list_modules)

# Run each (sequence of) module(s) with the desired system
# One instance of modules to process looks like that: [['SystemName1', [<Module1-Object>, <Module2-Object>]], ['SystemName2', [<Module3-Object>, <Module4-Object>]], ...]
# globals()[function_name] allow for calling a function using a string
for system_modules in modules_to_process:
  function_name = 'process_files_'+system_modules[0]
  print('--------------------------')
  print('Running '+system_modules[0])
  print('--------------------------')
  if len(system_modules[1]) > 1:
    if group_modules_prm == 'yes':
      grouped_module_object = create_grouped_module_objects_FORGe(system_modules[1])
      globals()[function_name](grouped_module_object)
    else:
      for module_object in system_modules[1]:
        print("Processing module #"+str(system_modules[1].index(module_object)))
        globals()[function_name](module_object)
  else:
    globals()[function_name](system_modules[1][0])

print('=================================================')
print('All done!')

openjdk version "1.8.0_382"
OpenJDK Runtime Environment (build 1.8.0_382-8u382-ga-1~22.04.1-b05)
OpenJDK 64-Bit Server VM (build 25.382-b05, mixed mode)
Clearing output folders...
Preparing generation pipeline...
  -> Initial structure is ['PredArg'].
  -> 10 modules were selected.
  -> Sequence: ['PredArgNorm', 'PredArgAgg', 'PredArgPoS', 'PredArgComm', 'DSynt', 'SSynt', 'SSyntAgg', 'REG', 'DMorphLin', 'SMorphText']
  -> The pipeline looks good, proceeding...
--------------------------
Running FORGe
--------------------------
  Cleared input folder...
  Generated mate.properties file...
  Copied files to input folder...
  Renamed files in input folder...
  Running module(s)...
Loading files...
Parsing 10_Con_Sem.rl...
Parsing 11.1_Con_Agg1.rl...
Parsing 11.2_Con_Agg2.rl...
Parsing 11.3_Con_Agg3.rl...
Parsing 11.4_Con_Agg4.rl...
Parsing 13_Sem_SemPoS.rl...
Parsing 15_SemPoS_SemCommMark.rl...
Parsing 17_SemCommMark_SemComm.rl...
Parsing 20_SemComm_DSynt.rl...
Parsing 30_DSynt_SSynt.rl..

## FORGe check outputs and copy to Morph input folder

In [6]:
input_folder = '/content/FORGe/structures/00-PredArg'
output_folder = '/content/FORGe/structures/11-SMorphText'
input_folder_morph = '/content/test_irish_morph_gen_v5.0/FORGe-out'

str_count_perLevel = []
txt_count_perLevel = []

def count_conll(filePath):
  """ counts how many conll structures are in a file """
  counter = 0
  fd = codecs.open(filePath, 'r', 'utf-8')
  lines = fd.readlines()
  for line in lines:
    if re.search('^0\t_', line):
      counter += 1
  return(counter)

def count_txt(filePath):
  """ counts how many texts are in a file """
  fd = codecs.open(filePath, 'r', 'utf-8')
  lines = fd.readlines()
  return(len(lines))

def examine_files(path, count_perLevel):
  folder_content = os.listdir(path)
  # Sorting the files so they remain aligned with the outputs
  for file in sorted(folder_content):
    file_path = os.path.join(path, file)
    # If files are found in the folder, process them
    # Get number of input structures
    if os.path.isfile(file_path):
      if re.search('\.conll', file_path):
        count = count_conll(file_path)
        count_perLevel.append(count)
    # If folders are found in the folder, go deeper
    elif os.path.isdir(file_path):
      str_subfolder_content = os.listdir(file_path)
      # Sorting the folders so they remain aligned with the inputs
      for deeper_content in sorted(str_subfolder_content):
        new_file_path = os.path.join(file_path, deeper_content)
        # Get number of texts
        if os.path.isfile(new_file_path):
          if re.search('\.txt', new_file_path):
            count = count_txt(new_file_path)
            count_perLevel.append(count)
            shutil.copy(new_file_path, input_folder_morph)

examine_files(input_folder, str_count_perLevel)
examine_files(output_folder, txt_count_perLevel)


if str_count_perLevel == txt_count_perLevel:
  print('OK!\n----')
else:
  print('Problem!\n----')

print('Inputs:  ' + str(sum(str_count_perLevel)))
print('Outputs: ' + str(sum(txt_count_perLevel)))

print('Inputs:  ' + str(str_count_perLevel))
print('Outputs: ' + str(txt_count_perLevel))


OK!
----
Inputs:  9
Outputs: 9
Inputs:  [9]
Outputs: [9]


# 3 - Morphology processing (Irish NLP Tools)

## Process raw FORGe output

In [7]:
import glob
import codecs
import re
import os

# I copy the FORGe outputs manually to this file
# filepath = os.path.join(folder_name,'aerfort_gen_test.txt')
# filepath = os.path.join(folder_name,'test_triples_en_ga_utf8_0000-0449.conll_out.txt')
# filepath = os.path.join(folder_name,'test_triples_en_ga_utf8_0450-0899.conll_out.txt')
# filepath = os.path.join(folder_name,'test_triples_en_ga_utf8_0900-1349.conll_out.txt')
# filepath = os.path.join(folder_name,'test_triples_en_ga_utf8_1350-1778.conll_out.txt')

def process_FORGe(filepath, count_strs_all):
  filename = filepath.rsplit('/', 1)[1]
  print('Processing '+filename)
  lines = codecs.open(filepath, 'r', 'utf-8').readlines()
  fo = codecs.open(os.path.join(morph_input_folder, filename), 'w', 'utf-8')
  for line in lines:
    # Replace apostrophes/parentheses by something else, they make morph break
    line = re.subn("'", '_APSTR_', line)[0]
    line = re.subn("\(", '_OBRKT_', line)[0]
    line = re.subn("\)", '_CBRKT_', line)[0]
    line = re.subn("&", '_AMPRS_', line)[0]
    line = re.subn(";", '_SEMICOL_', line)[0]
    line = re.subn("\$", '_DOLLSIGN_', line)[0]
    line = re.subn("\+", '_PLUSSIGN_', line)[0]
    line = re.subn('\"', '_DBLQUOT_', line)[0]
    # clean quotes that MATE can’t take care of
    line = re.subn('\\\\"([^\\\\]+)\\\\', '\\"\g<1>\\"', line)[0]
    # In FORGe I use % as separator, we need +
    line = re.subn('%', '+', line)[0]
    # Insert linebreak after each final sentence to separate texts from each other (one text can have several sentences)
    line = re.subn(' \.$', ' .\n', line)[0]
    # Put each word on one line
    line = re.subn(' ', '\n', line)[0]
    # Copy to morph gen input folder
    fo.write(line)
  fo.close()
  count_strs_all.append(len(lines))

# list_filepaths = glob.glob(os.path.join(folder_name, 'FORGe-out', '*conll_out.txt'))
list_filepaths = glob.glob(os.path.join(folder_name, 'FORGe-out', '*.txt'))

# To store how many texts we have in each file (used to )
count_strs_all_FORGe = []

for filepath in list_filepaths:
  process_FORGe(filepath, count_strs_all_FORGe)

# Check
print('\nThere are '+str(sum(count_strs_all_FORGe))+' texts.')
print(count_strs_all_FORGe)

Processing 230823_dev_4tr_sml_GA__SMorphText.conll_out.txt

There are 9 texts.
[9]


## Call morph generator

In [8]:
# v3 (fast, ~2sec/450 texts)

# Run the morphology generation
from IPython.display import HTML, display
import progressbar
import glob
import codecs
from termcolor import colored

show_input = False #@param {type:"boolean"}

# To store how many texts we have in each file (used to )
count_strs_all_Morph = []

for filepath in glob.glob(os.path.join(morph_input_folder, '*.*')):
  count_strs_all = 0
  head, tail = os.path.split(filepath)
  filename = tail.rsplit('.')[0]
  print('Processing '+filename)
  fo = codecs.open(morph_output_folder+'/'+filename+'_out.txt', 'w', 'utf-8')
  list_inflected_words = ! cat {filepath} | {folder_name}'/flookup' -a {folder_name}'/allgen.fst'
  # print(list_inflected_words)

  # Create a variable to store the outputs
  text = ''
  # morph returns this as list_inflected_words: ['imir+Verb+Vow+PresInd\timríonn', '', 'Agremiação_Sportiva_Arapiraquense+Noun+Masc+Com+Pl\t+?', '', ',\t+?',...]
  for word in list_inflected_words:
    empty = 'yes'
    input_string = ''
    morph_returned = ''
    morph_backup = ''
    if re.search('\t', word):
      # for every space an empty string is returned; we'll ignore them later. Between two consecutive texts there is a simple "\t" with nothing around. I use this to introduce linebreaks later.
      empty = 'no'
      input_string = word.split('\t')[0]
      morph_returned = word.split('\t')[1]
      if re.search('\+', word):
        morph_backup = input_string.split('+', 1)[0]
      else:
        morph_backup = input_string
    out_line = ''
    # Create each output line with the required contents
    if show_input == True:
      if empty == 'no':
        if morph_returned == '':
          if input_string == '':
            out_line = out_line + '\n'
            count_strs_all += 1
        else:
          out_line = out_line + input_string + ': ' +'\x1b[5;30;47m'+morph_returned+'\x1b[0m'+'\n'
    else:
      if empty == 'no':
        if morph_returned == '+?':
          out_line = out_line + morph_backup + ' '
        # If the line is empty, add a line break (empty lines separate different texts in the input)
        elif morph_returned == '':
          if input_string == '':
            out_line = out_line + '\n'
            count_strs_all += 1
        else:
          out_line = out_line + morph_returned + ' '
    # add line to the other lines of the same file
    text = text + out_line

  # print('\n----------------------\n'+text+'\n')
  count_strs_all_Morph.append(count_strs_all)
  fo.write(text+'\n')
  fo.close()

# Check
print('\nThere are '+str(sum(count_strs_all_Morph))+' texts.')
print(count_strs_all_Morph)
if not sum(count_strs_all_Morph) == sum(count_strs_all_FORGe):
  print('\nERROR! Mismatch with FORGe outputs!')

Processing 230823_dev_4tr_sml_GA__SMorphText

There are 9 texts.
[9]


# 4 - Output post-processing and packaging

In [9]:
# Clean output text

import glob
import sys
import os
import codecs
import re

#encodings: cp1252, utf-8, utf-16BE ...

#get path of input folder
containing_path = morph_output_folder.rsplit('\\', 1)[0].replace('\\', '/')
language = 'GA'
with_underscores = 'no'
without_underscores = 'yes'
#folder_name = path.rsplit('\\', 1)[1]

#filename = os.path.join(containing_path, str(x)+'_report.txt')

#create output file
#fo = codecs.open(filename, 'a', 'utf-8')

#read filepaths
#list_filepaths = glob.glob(os.path.join(path, '*.conll'))
list_filepaths = glob.glob(os.path.join(morph_output_folder, '*_out.txt'))

def uppercase(match):
  return(match.group(1).upper())

def uppercaseDot(match):
  return('. '+match.group(1).upper())

def clean_outputs (text, count, underscores):
  # Irish things
  if language == 'GA':
    # Reestablish punctuations removed for morph processing
    text = re.subn("_APSTR_", "'", text)[0]
    text = re.subn("_OBRKT_", "(", text)[0]
    text = re.subn("_CBRKT_", ")", text)[0]
    text = re.subn("_AMPRS_", "&", text)[0]
    text = re.subn("_SEMICOL_", ";", text)[0]
    text = re.subn("_DOLLSIGN_", "$", text)[0]
    text = re.subn("_PLUSSIGN_", "+", text)[0]
    text = re.subn("_DBLQUOT_", '\"', text)[0]
    # Remove tig/thig (bug morph, fixed)
    # text = re.subn("tagann tig ", "tagann ", text)[0]
    # text = re.subn("thagann thig ", "thagann ", text)[0]
    # Process prefixes (either after a space or at the beginning of a line)
    text = re.subn(" d- ", " d'", text)[0]
    text = re.subn("^d- ", "d'", text)[0]
    text = re.subn(' h- ', ' h', text)[0]
    text = re.subn('^h- ', 'h', text)[0]
    text = re.subn(' t- ', ' t-', text)[0]
    text = re.subn('^t- ', 't-', text)[0]
    # The hyphens left should all be from prefixed adjectives
    text = re.subn('- - ', '', text)[0]
    text = re.subn(' - ', '', text)[0]
    text = re.subn('- ', '', text)[0]
    # Bring together a + tá in relative clauses
    text = re.subn(' a tá ', ' atá ', text)[0]
    # contract "de" in front of "fh" and vowels, i, etc.
    text = re.subn(' de [aA]n ', ' den ', text)[0]
    text = re.subn(' de [aA]n_', ' den ', text)[0]
    text = re.subn(' do [aA]n ', ' don ', text)[0]
    text = re.subn(' do [aA]n_', ' don ', text)[0]
    text = re.subn(' faoi [aA]n ', ' faoin ', text)[0]
    text = re.subn(' faoi [aA]n_', ' faoin ', text)[0]
    text = re.subn(' ó [aA]n ', ' ón ', text)[0]
    text = re.subn(' ó [aA]n_', ' ón ', text)[0]
    text = re.subn(" d[eo] ([fF])h", " d'\g<1>h", text)[0]
    text = re.subn(" de ([aeiouAEIOUáéíóúÁÉÍÓÚ])", " d'\g<1>", text)[0]
    text = re.subn(' i Éire ', ' in Éirinn ', text)[0]
    text = re.subn(' i An ', ' ins An ', text)[0]
    text = re.subn(' i An_', ' ins An ', text)[0]
    text = re.subn(' i ([aeiouáéíóúAEIOUÁÉÍÓÚ])', ' in \g<1>', text)[0]
    # Lenition f (+ contraction)
    text = re.subn(' d[eo] ([fF])([^h])', " d'\g<1>h\g<2>", text)[0]
    text = re.subn(" (ar|de|do|faoi|mar|ó|roimh|trí|um|céad) ([bcdfgmptBCDFGMPT])([^hH])", " \g<1> \g<2>h\g<3>", text)[0]
    text = re.subn(" (ar|de|do|faoi|mar|ó|roimh|trí|um|céad) ([sS])([^hcfmptvHCFMPTV])", " \g<1> \g<2>h\g<3>", text)[0]
    # Eclipsis
    text = re.subn(' i ([bB][^pP])', ' i m\g<1>', text)[0]
    text = re.subn(' i ([cC])', ' i g\g<1>', text)[0]
    text = re.subn(' i ([dD][^tT])', ' i n\g<1>', text)[0]
    text = re.subn(' i ([fF])', ' i bh\g<1>', text)[0]
    text = re.subn(' i ([gG][^cC])', ' i n\g<1>', text)[0]
    text = re.subn(' i ([pP])', ' i b\g<1>', text)[0]
    text = re.subn(' i ([tT])', ' i d\g<1>', text)[0]
    # text = re.subn(' i ([aeiouáéíóú])', ' i n-\g<1>', text)[0]
    # text = re.subn(' i ([AEIOUÁÉÍÓÚ])', ' i n\g<1>', text)[0]
    # Ugly patches
    text = re.subn(' ar bhí ', ' a bhí ', text)[0]
    text = re.subn(' an ann ', ' air ', text)[0]
    text = re.subn(' an sé ', ' an é ', text)[0]
  # Erroneous "type = parenthetical" fix
  if re.search('\) \(', text):
    print('!!! Failed parenthesis generation in input '+str(count)+' (fixed)')
    text = re.subn('\) \(', ' ', text)[0]
  # clean quotes that MATE can’t take care of
  text = re.subn('\\\\"([^\\\\]+)\\\\', '"\g<1>"', text)[0]
  # uppercase words when at the beginning of a sentence or after a final dot (dots preceded by a space)
  text = re.subn(' \. ([a-z])', uppercaseDot, text)[0]
  text = re.subn('^([a-z])', uppercase, text)[0]
  # replace "a" by "an" before vowels (should restrict to English)
  if language == 'EN':
    text = re.subn(' a ([aeio])', ' an \g<1>', text)[0]
  # find generation fails (we introduce [..] or [...] or [......] when a sentence cannot be generated
  if re.search('\[\.\.', text):
    print('!!! Failed sentence generation in input '+str(count))
  # remove space before commas and dots
  text = re.subn(' ,', ',', text)[0]
  text = re.subn(' \.', '.', text)[0]
  # replace double dots by single ones
  while re.search('\.\.', text):
    text = re.subn('\.\.', '.', text)[0]
  text = re.subn('% %', '%', text)[0]
  # Clean remnants of non-generated sentences
  text = re.subn('Sentence \[\.\]\.', '', text)[0]
  # Remove initial spaces
  while re.search('^ ', text):
    text = re.subn('^ ', '', text)[0]
  # replace underscores by spaces
  if underscores == 'yes':
    pass
  else:
    text = re.subn('_', ' ', text)[0]
  # New 2023: replace double spaces by single ones
  while re.search('  ', text):
    text = re.subn('  ', ' ', text)[0]
  # reformat date/time
  text = re.subn('([0-9]+)-([0-9]+)-([0-9]+)T([0-9]+:[0-9]+:[0-9]+)Z', '\g<1>/\g<2>/\g<3> at \g<4>', text)[0]
  return(text)

count_strs_all_postproc = []
for filepath in list_filepaths:
  count_strs_all = 0
  # filename = filepath.split('\\')[-1].split('.')[0]
  head, tail = os.path.split(filepath)
  filename = tail.rsplit('.')[0]
  fd = codecs.open(filepath, 'r', 'utf-8')
  print('Processing '+filename)
  filename_out_noUnderscores = filename+'_postproc''.txt'
  filename_out_Underscores = filename+'_postproc_underscores''.txt'
  if without_underscores == 'yes':
    fo1 = codecs.open(os.path.join(morph_output_folder, filename_out_noUnderscores), 'w', 'utf-8')
  if with_underscores == 'yes':
    fo2 = codecs.open(os.path.join(morph_output_folder, filename_out_Underscores), 'w', 'utf-8')
  lines = fd.readlines()
  x = 0
  for line in lines:
    # To filter out final linebreak in each file
    if not line == '\n':
      if without_underscores == 'yes':
        new_line = clean_outputs(line, x, 'no')
        fo1.write(new_line)
      if with_underscores == 'yes':
        new_line = clean_outputs(line, x, 'yes')
        fo2.write(new_line)
      count_strs_all += 1
    x += 1
  if without_underscores == 'yes':
    fo1.close()
  if with_underscores == 'yes':
    fo2.close()
  count_strs_all_postproc.append(count_strs_all)

# Check
print('\nThere are '+str(sum(count_strs_all_postproc))+' texts.')
print(count_strs_all_postproc)
if not sum(count_strs_all_postproc) == sum(count_strs_all_FORGe):
  print('\nERROR! Mismatch with FORGe outputs!')

Processing 230823_dev_4tr_sml_GA__SMorphText_out

There are 9 texts.
[9]


In [10]:
# Concatenate files

list_clean_outputs = glob.glob(os.path.join(morph_output_folder, '*_out_postproc.txt'))
filename = 'all_out.txt'

with codecs.open(filename, 'w', 'utf-8') as outfile:
  # Files need to be sorted to be concatenated in the right order
  for fname in sorted(list_clean_outputs):
    print('Processing '+fname)
    with open(fname) as infile:
      outfile.write(infile.read())

# Check
count_texts_all = len(codecs.open(filename).readlines())
print('\nThere are '+str(count_texts_all)+' texts.')
if not count_texts_all == sum(count_strs_all_FORGe):
  print('\nERROR! Mismatch with FORGe outputs!')

Processing /content/test_irish_morph_gen_v5.0/Outputs/230823_dev_4tr_sml_GA__SMorphText_out_postproc.txt

There are 9 texts.


## Zip output folder to download

In [11]:
split = "ukn" #@param['dev', 'test','train','ukn']
!zip -r /content/WebNLG_[{split}]_allLevels.zip /content/FORGe/structures

  adding: content/FORGe/structures/ (stored 0%)
  adding: content/FORGe/structures/08-SSyntAgg/ (stored 0%)
  adding: content/FORGe/structures/01-PredArgNorm/ (stored 0%)
  adding: content/FORGe/structures/03-PredArgAgg/ (stored 0%)
  adding: content/FORGe/structures/00-PredArg/ (stored 0%)
  adding: content/FORGe/structures/00-PredArg/230823_dev_4tr_sml_GA.conll (deflated 88%)
  adding: content/FORGe/structures/05-PredArgComm/ (stored 0%)
  adding: content/FORGe/structures/06-DSynt/ (stored 0%)
  adding: content/FORGe/structures/11-SMorphText/ (stored 0%)
  adding: content/FORGe/structures/11-SMorphText/230823_dev_4tr_sml_GA__SMorphText.conll/ (stored 0%)
  adding: content/FORGe/structures/11-SMorphText/230823_dev_4tr_sml_GA__SMorphText.conll/230823_dev_4tr_sml_GA__SMorphText.conll_out.txt (deflated 79%)
  adding: content/FORGe/structures/11-SMorphText/230823_dev_4tr_sml_GA__SMorphText.conll/230823_dev_4tr_sml_GA__SMorphText.conll_out.str (deflated 84%)
  adding: content/FORGe/structu